In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.3
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 3) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(epochs = 300000, npoints = 9, batchsize = 4000, nsamples = 8) #greedyprob - 3, cutoff for negative is greedy performance + 0.3

15.020000457763672 -4.351398468017578 -4.762119770050049 -4.40485954284668 -4.435622692108154 -4.435622692108154
14.29525089263916 -4.139331340789795 -4.716104984283447 -4.219272613525391 -5.555254936218262 -5.555253982543945
13.837250709533691 -3.5807688236236572 -4.501070976257324 -3.427811622619629 -3.7871437072753906 -3.7871437072753906
13.662750244140625 -2.959322452545166 -4.10951566696167 -2.4176547527313232 -3.427647113800049 -3.427647113800049
13.584250450134277 -2.809297561645508 -3.989410638809204 -2.433485507965088 -4.057412624359131 -4.057413101196289
13.682750701904297 -2.7024927139282227 -3.8650412559509277 -3.0726161003112793 -3.773671865463257 -3.7736713886260986
13.587250709533691 -2.512638568878174 -3.6638777256011963 -3.8925158977508545 -4.200430870056152 -4.200430870056152
13.512750625610352 -2.187556743621826 -3.3823189735412598 -2.065944194793701 -3.1965830326080322 -3.1965832710266113
13.35425090789795 -1.967801570892334 -3.190971612930298 -2.2982616424560547 -4

12.473000526428223 -1.1522396802902222 -2.199073314666748 -1.609100341796875 -1.0394139289855957 -1.0394141674041748
12.571250915527344 -1.1475083827972412 -2.180304765701294 -0.37840354442596436 -1.6920832395553589 -1.6920832395553589
12.43850040435791 -1.1271929740905762 -2.1608622074127197 -1.6895443201065063 -1.8300806283950806 -1.830080509185791
12.440500259399414 -1.1098843812942505 -2.163424015045166 -1.034879446029663 -1.196337103843689 -1.1963368654251099
12.441750526428223 -1.1051459312438965 -2.1233956813812256 -0.5945786237716675 -1.116417407989502 -1.116417646408081
12.491250991821289 -1.1366022825241089 -2.1701860427856445 -0.3102078437805176 -0.24913811683654785 -0.24913811683654785
12.444750785827637 -1.1335293054580688 -2.177239179611206 -1.2912404537200928 -2.962136745452881 -2.9621362686157227
12.464500427246094 -1.1312953233718872 -2.1883044242858887 -0.6863851547241211 -0.6440277099609375 -0.6440277099609375
12.475000381469727 -1.122684359550476 -2.186694860458374 

12.243000984191895 -1.0234556198120117 -1.9803270101547241 -1.713552713394165 -2.7119412422180176 -2.7119412422180176
12.294750213623047 -1.009472370147705 -1.979539155960083 -0.8090629577636719 -0.1132669448852539 -0.1132669448852539
12.27125072479248 -1.022590160369873 -1.966719388961792 -0.7372833490371704 -6.013741970062256 -6.013742446899414
12.238000869750977 -1.0045233964920044 -1.9386160373687744 -1.7018721103668213 -2.442505359649658 -2.442505359649658
12.266000747680664 -0.9938629865646362 -1.9268347024917603 -0.7906243205070496 -1.7960623502731323 -1.7960624694824219
12.245250701904297 -0.9909637570381165 -1.9489740133285522 -1.7131047248840332 -0.79271399974823 -0.7927142381668091
12.257750511169434 -0.9887524247169495 -1.9282524585723877 -0.3467335104942322 -1.5289345979690552 -1.5289348363876343
12.259500503540039 -0.969905436038971 -1.9107110500335693 -0.7661751508712769 -1.033650517463684 -1.0336503982543945
12.315500259399414 -0.998242199420929 -1.9277751445770264 -1.6

12.159250259399414 -0.9134118556976318 -1.7876582145690918 -1.8044945001602173 -1.9794872999191284 -1.9794871807098389
12.168000221252441 -0.9134091138839722 -1.7822600603103638 -0.40642741322517395 -0.7479608058929443 -0.7479609251022339
12.221250534057617 -0.9027405381202698 -1.7433722019195557 -0.45612749457359314 -1.5639393329620361 -1.5639392137527466
12.124250411987305 -0.9087451696395874 -1.7647624015808105 -0.5194759368896484 -4.878085136413574 -4.878085136413574
12.132250785827637 -0.9068796634674072 -1.775002121925354 -0.28099751472473145 -1.09404456615448 -1.094044804573059
12.15000057220459 -0.920985996723175 -1.8019577264785767 -0.4398151636123657 -2.4021692276000977 -2.4021694660186768
12.209000587463379 -0.9319032430648804 -1.817452311515808 -0.2880939841270447 -1.1065388917922974 -1.106539011001587
12.160250663757324 -0.9324426651000977 -1.809006690979004 -0.843360185623169 -0.9011967182159424 -0.9011967182159424
12.147500991821289 -0.9320515990257263 -1.822368025779724

12.107000350952148 -0.9290604591369629 -1.8000166416168213 -0.29278862476348877 -3.96498703956604 -3.964986801147461
12.152750968933105 -0.9021715521812439 -1.779926061630249 -0.758162260055542 -0.8957479000091553 -0.8957479000091553
12.15475082397461 -0.8670940399169922 -1.740738034248352 -0.7140259742736816 -0.4371521472930908 -0.4371525049209595
12.062500953674316 -0.8791100978851318 -1.7313209772109985 -0.8658841848373413 -0.39788615703582764 -0.39788615703582764
12.199750900268555 -0.8620371222496033 -1.7116755247116089 -1.2405050992965698 -1.6851632595062256 -1.685163140296936
12.101250648498535 -0.8698882460594177 -1.7216565608978271 -0.6947827339172363 -2.742093086242676 -2.742093086242676
12.150250434875488 -0.8816027641296387 -1.7341524362564087 -0.8129259943962097 -2.296189785003662 -2.296189546585083
12.230250358581543 -0.894487202167511 -1.7463628053665161 -0.6605145335197449 -1.4331839084625244 -1.433184027671814
12.171250343322754 -0.895696759223938 -1.7336492538452148 -

...

11.397250175476074 -0.34058716893196106 -0.6975553631782532 -0.1379716396331787 -0.06449365615844727 -0.06449365615844727
11.451000213623047 -0.35912907123565674 -0.7270161509513855 -0.001020193099975586 -0.07654571533203125 -0.07654571533203125
11.468750953674316 -0.35807090997695923 -0.7172675132751465 -0.154860258102417 -0.3652966022491455 -0.3652963638305664
11.456750869750977 -0.35246211290359497 -0.7091507315635681 -0.6975991725921631 -0.1317431926727295 -0.1317434310913086
11.459000587463379 -0.34012988209724426 -0.6969092488288879 -0.4765644073486328 -3.640742301940918 -3.640742778778076
11.454751014709473 -0.34189608693122864 -0.6935620903968811 -0.004396915435791016 -0.37537717819213867 -0.37537717819213867
11.431000709533691 -0.35315656661987305 -0.693244218826294 -0.29880475997924805 -0.9853909015655518 -0.9853918552398682
11.443750381469727 -0.34104683995246887 -0.6910309195518494 -0.2349429726600647 -0.288989782333374 -0.288989782333374
11.44575023651123 -0.34373417496681

11.483000755310059 -0.34365394711494446 -0.7104410529136658 -0.5273923873901367 -0.3704240322113037 -0.3704237937927246
11.429000854492188 -0.34158965945243835 -0.7042269110679626 -0.3684213161468506 -0.41979849338531494 -0.41979849338531494
11.506000518798828 -0.3436352014541626 -0.7101244330406189 -0.024709224700927734 -0.03805696964263916 -0.03805696964263916
11.411750793457031 -0.33972030878067017 -0.6913688778877258 -0.4726121425628662 -0.06611812114715576 -0.06611812114715576
11.435500144958496 -0.3300130069255829 -0.6953153014183044 -0.29037177562713623 -0.10131669044494629 -0.1013176441192627
11.472250938415527 -0.3448108732700348 -0.6908006072044373 -0.029540538787841797 -0.269578218460083 -0.2695784568786621
11.490500450134277 -0.3509428799152374 -0.6915233135223389 -0.024692535400390625 -0.030453205108642578 -0.03045368194580078
11.47350025177002 -0.3409109115600586 -0.6892822980880737 -0.17118966579437256 -0.003993988037109375 -0.003993988037109375
11.450000762939453 -0.348

11.488000869750977 -0.3549533188343048 -0.7367729544639587 -0.0013203620910644531 -0.03711366653442383 -0.03711366653442383
11.382750511169434 -0.350391685962677 -0.7124218940734863 -0.18845701217651367 -0.07652473449707031 -0.07652473449707031
11.433000564575195 -0.3434205949306488 -0.713658332824707 -0.1036825180053711 -0.3121771812438965 -0.3121771812438965
11.456000328063965 -0.34668058156967163 -0.7125982046127319 -0.9213710427284241 -1.9769799709320068 -1.9769797325134277
11.491250991821289 -0.3380161225795746 -0.7153268456459045 -0.11339235305786133 -0.5810065269470215 -0.5810065269470215
11.465750694274902 -0.35125067830085754 -0.7010998725891113 -0.36257076263427734 -0.7940616607666016 -0.7940616607666016
11.472250938415527 -0.35627803206443787 -0.7092574834823608 -0.5233283042907715 -2.775087356567383 -2.775088310241699
11.447750091552734 -0.3535359799861908 -0.7193871140480042 -0.009493827819824219 -0.15112829208374023 -0.15112781524658203
11.403250694274902 -0.3457386493682

11.41100025177002 -0.34212526679039 -0.6993251442909241 -0.05809807777404785 -0.1029958724975586 -0.1029958724975586
11.406500816345215 -0.3494122624397278 -0.7102351188659668 -0.5041663646697998 -0.04685068130493164 -0.04685068130493164
11.483500480651855 -0.35396772623062134 -0.7116596102714539 -0.0008931159973144531 -0.6981418132781982 -0.6981415748596191
11.456501007080078 -0.349021315574646 -0.7066410183906555 -0.3040437698364258 -0.00653231143951416 -0.00653231143951416
11.488000869750977 -0.36215493083000183 -0.7189471125602722 -0.7064270973205566 -0.07387280464172363 -0.07387280464172363
11.48425006866455 -0.3474881649017334 -0.7223328351974487 -0.03879737854003906 -0.013689994812011719 -0.013689994812011719
11.500000953674316 -0.35104456543922424 -0.7047311663627625 -0.15749168395996094 -0.06553792953491211 -0.06553792953491211
11.410000801086426 -0.3397377133369446 -0.7083040475845337 -0.6257559061050415 -1.511508822441101 -1.511508584022522
11.459750175476074 -0.352381557226

11.456501007080078 -0.3467193841934204 -0.6974749565124512 -0.43224287033081055 -2.5601508617401123 -2.560150623321533
11.442000389099121 -0.35376477241516113 -0.7071176171302795 -1.1365511417388916 -0.03578805923461914 -0.03578805923461914
11.408500671386719 -0.34591418504714966 -0.7136795520782471 -0.366727739572525 -1.548722267150879 -1.5487215518951416
11.416000366210938 -0.33611413836479187 -0.6908349394798279 -0.0023152828216552734 -0.4190945625305176 -0.4190945625305176
11.412750244140625 -0.34338513016700745 -0.702136754989624 -0.6831676959991455 -0.00035834312438964844 -0.00035834312438964844
11.442750930786133 -0.34495988488197327 -0.6996785402297974 -0.034470558166503906 -0.12984347343444824 -0.12984347343444824
11.492000579833984 -0.3587862253189087 -0.7111237645149231 -0.27628493309020996 -0.07108211517333984 -0.07108211517333984
11.441750526428223 -0.33918213844299316 -0.6930306553840637 -0.044470787048339844 -0.30371320247650146 -0.30371344089508057
11.425000190734863 -0

In [3]:
train(epochs = 300000, npoints = 9, batchsize = 4000, nsamples = 8)

RuntimeError: CUDA out of memory. Tried to allocate 282.00 MiB (GPU 1; 15.90 GiB total capacity; 14.24 GiB already allocated; 237.38 MiB free; 14.77 GiB reserved in total by PyTorch)

In [2]:
torch.save(NN.state_dict(),'3D_5points_11.45_fast')